# Tensorflow Regression

In [1]:

fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [9]:

raw_dataset_name = 'raw_dataset_2022-10-18.csv'

df_raw_data = pd.read_csv(f'{fileloc}/{raw_dataset_name}')

print(df_raw_data.shape)
print()
print(df_raw_data.columns.values.tolist())


(245827, 12)

['@id_DOG', 'Place', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'x_dist', 'df_avg_speed_prev', 'x_races_tot']


In [10]:

df_raw_data.isna().sum()


@id_DOG                 0
Place                   0
DogName                 0
y_runtime            2445
TrainerName             0
@id_RACE                0
RaceNum                 0
RaceName                0
Track                   0
x_dist                  0
df_avg_speed_prev     191
x_races_tot             0
dtype: int64

In [12]:

'''
Predicted Variable 
    0.99% y_runtime missing =  2445 records
'''

print(df_raw_data.shape)
df_raw_data = df_raw_data[~df_raw_data.y_runtime.isna()]
print(df_raw_data.shape)


(245827, 12)
(243382, 12)


In [13]:

df_raw_data.isna().sum()


@id_DOG                0
Place                  0
DogName                0
y_runtime              0
TrainerName            0
@id_RACE               0
RaceNum                0
RaceName               0
Track                  0
x_dist                 0
df_avg_speed_prev    187
x_races_tot            0
dtype: int64

In [19]:

'''
df_avg_speed_prev is missing = first race
    create another variable for first race
    impute missing value as the average of runtime for [trainer, x_rac_tot = 1]
'''

#print( df_raw_data.x_races_tot.value_counts() )

df_raw_data['x_no_prev_races'] = df_raw_data.apply( lambda x : 1 if pd.isna(x.df_avg_speed_prev) else 0, axis = 1 )

print( df_raw_data['x_no_prev_races'].value_counts() )



0    243195
1       187
Name: x_no_prev_races, dtype: int64


In [38]:

'''
Imputing the average speed as the "average speed of ALL greyhounds on their first race at the track"
'''

#replace_group = ['TrainerName','x_races_tot']
#replace_group = ['Track','x_races_tot']
replace_group = ['Track']

df_mvt_avg_speed = df_raw_data[ (df_raw_data.x_races_tot == 0) & ~(df_raw_data.df_avg_speed_prev.isna()) ].groupby( replace_group ).agg( {'df_avg_speed_prev':'mean'} ).rename( columns = { 'df_avg_speed_prev' : 'avg_speed_impute' } )

print(df_raw_data.shape)

df_raw_data_impute = pd.merge( df_raw_data, df_mvt_avg_speed, on = replace_group, how = 'left')

df_raw_data_impute['x_avg_speed_prev'] = df_raw_data_impute.apply(lambda x : x.avg_speed_impute if pd.isna(x.df_avg_speed_prev) else x.df_avg_speed_prev, axis = 1)

print(df_raw_data_impute.shape)
print(df_raw_data_impute.columns.values.tolist())


(243382, 13)
(243382, 15)
['@id_DOG', 'Place', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'x_dist', 'df_avg_speed_prev', 'x_races_tot', 'x_no_prev_races', 'avg_speed_impute', 'x_avg_speed_prev']


In [39]:

df_raw_data_impute.isna().sum()


@id_DOG                0
Place                  0
DogName                0
y_runtime              0
TrainerName            0
@id_RACE               0
RaceNum                0
RaceName               0
Track                  0
x_dist                 0
df_avg_speed_prev    187
x_races_tot            0
x_no_prev_races        0
avg_speed_impute       1
x_avg_speed_prev       0
dtype: int64

In [30]:

df_raw_data_impute[df_raw_data_impute.df_avg_speed_prev.isna()]


,@id_DOG,Place,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,x_dist,df_avg_speed_prev,x_races_tot,x_no_prev_races,avg_speed_impute,x_avg_speed_prev
0,114244064,3,FOURTH DIMENTION,23.29,P Pollutro,747034592,4,KHANS HOSPITALITY SERVICES,Warragul,400,NaN,0,1,16929.863677,16929.863677
234,196374469,6,DIGGERS COVE,24.45,N Jackson,745616401,6,SANDAL WELDING,Rockhampton,407,NaN,0,1,16881.854774,16881.854774
2458,280075043,3,LEICA BOMBER,19.95,L Nichols,745290009,3,HAPPY BIRTHDAY TO ME!,Hobart,340,NaN,1,1,NaN,NaN
6033,312535830,8,NIPPY UNO,19.75,H Lin,752129611,8,SWANK'S LAWN & GARDENING,Temora,330,NaN,1,1,NaN,NaN
6264,313382514,6,GENERAL MAX,18.25,D Fennell,750654405,6,SWALLOW DRIVE MEATS (N/P) STAKE,Mount Gambier,305,NaN,1,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239794,662596055,7,SUNNYSIDE CLOVER,23.90,M Clark,826995691,2,PAUA KENNELS HT2,Warragul,400,NaN,1,1,NaN,NaN
240694,667648171,5,ON THE DECK,20.38,L Cahalan,819982724,2,THE DOGCAST MAIDEN STAKE,Angle Park,342,NaN,0,1,17052.656037,17052.656037
242287,678239779,1,INDY GO RUBY,23.21,J Wood,815100863,1,Qualifying Trial/Provincial/405,Mandurah,405,NaN,0,1,17051.562067,17051.562067
242464,685382530,1,STYLISH RICO,19.40,B Kline,822917301,2,TAB - VENUE MODE,Albion Park,331,NaN,1,1,NaN,NaN


In [ ]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']

# 'TrainerName', 'Track' have been dummified
cols_for_drop = [ 'df_avg_speed_prev' , 'avg_speed_impute' ]

todaydt = datetime.now().date()
exp_fname = f'modeling_dataset_{todaydt}.csv'

exp_mod_ds = df_raw_data_impute.drop(columns = cols_for_drop).rename(columns = { 'RunTime' : 'y_runtime' })


In [ ]:

print(exp_mod_ds.columns.values.tolist())


In [ ]:
exp_mod_ds

In [ ]:

exp_mod_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)
